In [2]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('DataPreparation').getOrCreate()

In [3]:
df = spark.read.csv('Trip.csv', inferSchema=True, header=True)
df.printSchema()

root
 |-- Make: string (nullable = true)
 |-- ManufactureYear: integer (nullable = true)
 |-- EngineModel: string (nullable = true)
 |-- DriverID: string (nullable = true)
 |-- StartTime: timestamp (nullable = true)
 |-- StopTime: timestamp (nullable = true)
 |-- DurationSec: integer (nullable = true)
 |-- DistanceKm: double (nullable = true)
 |-- FuelUsedL: double (nullable = true)
 |-- StartOdoKm: double (nullable = true)
 |-- IdleTimeSec: integer (nullable = true)
 |-- Loaded: integer (nullable = true)
 |-- Stops: integer (nullable = true)
 |-- MaxRPM: double (nullable = true)



In [4]:
#Remove invalid and extreme DurationSec observations
df2 = df.filter("DurationSec > 0").filter("DurationSec < 35000")
df2.count()

527991

In [5]:
#Remove invalid and extreme IdleTimeSec observations
df3 = df2.filter("IdleTimeSec > 0").filter("IdleTimeSec < 5000")
df3.count()

526058

In [6]:
#Remove invalid and extreme ManufactureYear observations
df4 = df3.filter("ManufactureYear > 2001").filter("ManufactureYear < 2020")
df4.count()

492497

In [7]:
df4.groupby("EngineModel").count().orderBy("count", ascending=False).show()

+-----------+------+
|EngineModel| count|
+-----------+------+
|     Type 6|298362|
|    Type 13|145289|
|     Type 9| 42957|
|     Type 5|  4187|
|    No Type|   921|
|     Type 4|   615|
|     Type 8|    76|
|     Type 7|    43|
|     Type 1|    16|
|     Type 2|    12|
|    Type 11|     6|
|    Type 12|     6|
|     Type 3|     4|
|    Type 10|     3|
+-----------+------+



In [8]:
from pyspark.sql.functions import col
df5 = df4.where(col("EngineModel").isin({"Type 6", "Type 13", "Type 5", "Type 4", "Type 8", "Type 7"}))
df5.groupby("EngineModel").count().orderBy("count", ascending=False).show()

+-----------+------+
|EngineModel| count|
+-----------+------+
|     Type 6|298362|
|    Type 13|145289|
|     Type 5|  4187|
|     Type 4|   615|
|     Type 8|    76|
|     Type 7|    43|
+-----------+------+



In [9]:
df5.groupby("Make").count().orderBy("count", ascending=False).show()

+-------+------+
|   Make| count|
+-------+------+
|No Make|422373|
| Make 1| 23807|
| Make 2|  1836|
| Make 3|   556|
+-------+------+



In [10]:
df6 = df5.filter("DistanceKm < 800").filter("DistanceKm > 0")
df6.count()

448369

In [11]:
df7 = df6.filter("FuelUsedL < 250")
df7.count()

448265

In [12]:
df8 = df7.filter("MaxRPM < 2500").filter("MaxRPM > 1250")
df8.count()

448173

In [13]:
df9 = df8.filter("Stops < 15")
df9.count()

438754

In [14]:
df10 = df9.drop("DriverID")
df11 = df10.drop("Loaded")
df11.printSchema()

root
 |-- Make: string (nullable = true)
 |-- ManufactureYear: integer (nullable = true)
 |-- EngineModel: string (nullable = true)
 |-- StartTime: timestamp (nullable = true)
 |-- StopTime: timestamp (nullable = true)
 |-- DurationSec: integer (nullable = true)
 |-- DistanceKm: double (nullable = true)
 |-- FuelUsedL: double (nullable = true)
 |-- StartOdoKm: double (nullable = true)
 |-- IdleTimeSec: integer (nullable = true)
 |-- Stops: integer (nullable = true)
 |-- MaxRPM: double (nullable = true)



In [15]:
df12 = df11.na.drop()
df12.count()

438754

In [16]:
df12.printSchema()

root
 |-- Make: string (nullable = true)
 |-- ManufactureYear: integer (nullable = true)
 |-- EngineModel: string (nullable = true)
 |-- StartTime: timestamp (nullable = true)
 |-- StopTime: timestamp (nullable = true)
 |-- DurationSec: integer (nullable = true)
 |-- DistanceKm: double (nullable = true)
 |-- FuelUsedL: double (nullable = true)
 |-- StartOdoKm: double (nullable = true)
 |-- IdleTimeSec: integer (nullable = true)
 |-- Stops: integer (nullable = true)
 |-- MaxRPM: double (nullable = true)



In [17]:
df12.groupby("EngineModel").count().orderBy("count", ascending=False).show()

+-----------+------+
|EngineModel| count|
+-----------+------+
|     Type 6|291746|
|    Type 13|142225|
|     Type 5|  4085|
|     Type 4|   611|
|     Type 8|    64|
|     Type 7|    23|
+-----------+------+



In [18]:
from pyspark.sql.functions import lit
type6 = df12.where(col("EngineModel").isin({"Type 6"})).withColumn("EngineModelNum",lit(6))
type6.count()

291746

In [19]:
type13 = df12.where(col("EngineModel").isin({"Type 13"})).withColumn("EngineModelNum",lit(13))
type13.count()

142225

In [20]:
type5 = df12.where(col("EngineModel").isin({"Type 5"})).withColumn("EngineModelNum",lit(5))
type5.count()

4085

In [21]:
type4 = df12.where(col("EngineModel").isin({"Type 4"})).withColumn("EngineModelNum",lit(4))
type4.count()

611

In [22]:
type8 = df12.where(col("EngineModel").isin({"Type 8"})).withColumn("EngineModelNum",lit(8))
type8.count()

64

In [23]:
type7 = df12.where(col("EngineModel").isin({"Type 7"})).withColumn("EngineModelNum",lit(7))
type7.count()

23

In [24]:
type13_2 = type13.sample(True, 291746/142225, 100)
type13_2.count()

291651

In [25]:
type5_2 = type5.sample(True, 291746/4085, 100)
type5_2.count()

291460

In [26]:
type4_2 = type4.sample(True, 291746/611, 100)
type4_2.count()

291934

In [27]:
type8_2 = type8.sample(True, 291746/64, 100)
type8_2.count()

292418

In [28]:
type7_2 = type7.sample(True, 291746/23, 100)
type7_2.count()

291542

In [29]:
dfAll = type6.union(type13_2).union(type5_2).union(type4_2).union(type8_2).union(type7_2)
dfAll.groupby("EngineModel").count().orderBy("count", ascending=False).show()

+-----------+------+
|EngineModel| count|
+-----------+------+
|     Type 8|292418|
|     Type 4|291934|
|     Type 6|291746|
|    Type 13|291651|
|     Type 7|291542|
|     Type 5|291460|
+-----------+------+



In [30]:
dfAll2 = dfAll.select('*', (dfAll.DistanceKm/dfAll.FuelUsedL).alias('FuelEfficiency'))
dfAll2.head()

Row(Make='No Make', ManufactureYear=2015, EngineModel='Type 6', StartTime=datetime.datetime(2019, 7, 27, 4, 39, 15), StopTime=datetime.datetime(2019, 7, 27, 6, 44, 56), DurationSec=7541, DistanceKm=200.47, FuelUsedL=66.98, StartOdoKm=963748.96, IdleTimeSec=74, Stops=2, MaxRPM=1486.0, EngineModelNum=6, FuelEfficiency=2.9929829799940277)

In [31]:
from pyspark.sql.types import IntegerType
dfAll2 = dfAll2.withColumn("EngineModelNum", dfAll2["EngineModelNum"].cast(IntegerType()))
dfAll2.printSchema()

root
 |-- Make: string (nullable = true)
 |-- ManufactureYear: integer (nullable = true)
 |-- EngineModel: string (nullable = true)
 |-- StartTime: timestamp (nullable = true)
 |-- StopTime: timestamp (nullable = true)
 |-- DurationSec: integer (nullable = true)
 |-- DistanceKm: double (nullable = true)
 |-- FuelUsedL: double (nullable = true)
 |-- StartOdoKm: double (nullable = true)
 |-- IdleTimeSec: integer (nullable = true)
 |-- Stops: integer (nullable = true)
 |-- MaxRPM: double (nullable = true)
 |-- EngineModelNum: integer (nullable = false)
 |-- FuelEfficiency: double (nullable = true)



In [32]:
dfAll3 = dfAll2.drop("Make")
dfAll3.printSchema()

root
 |-- ManufactureYear: integer (nullable = true)
 |-- EngineModel: string (nullable = true)
 |-- StartTime: timestamp (nullable = true)
 |-- StopTime: timestamp (nullable = true)
 |-- DurationSec: integer (nullable = true)
 |-- DistanceKm: double (nullable = true)
 |-- FuelUsedL: double (nullable = true)
 |-- StartOdoKm: double (nullable = true)
 |-- IdleTimeSec: integer (nullable = true)
 |-- Stops: integer (nullable = true)
 |-- MaxRPM: double (nullable = true)
 |-- EngineModelNum: integer (nullable = false)
 |-- FuelEfficiency: double (nullable = true)



In [33]:
dfAll4 = dfAll3.drop("FuelUsedL")
dfAll5 = dfAll4.drop("EngineModel")
dfAll5.printSchema()

root
 |-- ManufactureYear: integer (nullable = true)
 |-- StartTime: timestamp (nullable = true)
 |-- StopTime: timestamp (nullable = true)
 |-- DurationSec: integer (nullable = true)
 |-- DistanceKm: double (nullable = true)
 |-- StartOdoKm: double (nullable = true)
 |-- IdleTimeSec: integer (nullable = true)
 |-- Stops: integer (nullable = true)
 |-- MaxRPM: double (nullable = true)
 |-- EngineModelNum: integer (nullable = false)
 |-- FuelEfficiency: double (nullable = true)



In [34]:
dfAll5.agg({"FuelEfficiency": "max"}).collect()[0][0]

67700.0

In [35]:
dfAll5.filter("FuelEfficiency > 10").count()

50

In [36]:
dfAll6 = dfAll5.filter("FuelEfficiency < 10")

In [58]:
from pyspark.sql.functions import hour
final = dfAll6.select('ManufactureYear'
                      ,hour('StartTime').alias('StartTimeHour')
                      ,hour('StopTime').alias('StopTimeHour')
                      ,'DurationSec'
                      ,'StartOdoKm'
                      ,'IdleTimeSec'
                      ,'Stops'
                      ,'MaxRPM'
                      ,'EngineModelNum'
                      ,col('FuelEfficiency').alias('label'))